<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Content Based Personalization
## LightGBM on Azure Databricks<br>
This notebook provides a quick example of how to train LightGBM model on Azure Databricks and deploy it using MML Spark for a content personalization scenario.<br><br>
LightGBM \[1\] is a gradient boosting framework that uses tree-based learning algorithms.<br>
MML Spark \[2\] allows LightGBM to be called in a Spark environment which provides several advantages:
- Distributed computation for model development
- Easy integration into existing Spark workflows
- Model serving through Spark Serving \[3\]

## Global Settings and Imports

In [1]:
import os
import sys
from tempfile import TemporaryDirectory
sys.path.append("../../")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import requests

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
PySpark version: 2.3.1


In [2]:
# Setup MML Spark
if not is_databricks():
    spark = start_or_get_spark(packages=['Azure:mmlspark:0.16'])

from mmlspark import ComputeModelStatistics
from mmlspark import DiscreteHyperParam
from mmlspark import HyperparamBuilder
from mmlspark import LightGBMClassifier
from mmlspark import RandomSpace
from mmlspark import RangeHyperParam
from mmlspark import TuneHyperparameters

## Data Preparation
The Criteo Display Advertising Challenge (DAC) dataset [3] is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows and provides a good challenge for scalable machine learning. The dataset has a binary label column and 38 feature columns, where 13 columns are integer values (int00-int12) and 25 columns are categorical features (cat00-cat24). There is also a sample data consist of 100,000 rows which can be used by setting DATA_SIZE = 'sample'.<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and indicates a user interaction with an item, so this is a useful dataset to demonstrate how to build a model that will predict likelihood of a user interacting with an item based on the user context and item content features.


In [4]:
DATA_SIZE = 'sample'

In [11]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, local_cache_path='../..')

8.79MB [00:08, 1.04MB/s]                            


### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The FeatureHasher transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality which can be used effectively by LightGBM.<br><br>
Lastly the dataset is split randomly for training and testing the model.

In [12]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')
data = feature_processor.transform(raw_data)
train, test = spark_random_split(data, ratio=0.8, seed=42)

In [13]:
# save data to tables to finalize feature transformation
train.write.mode('overwrite').saveAsTable('train')
test.write.mode('overwrite').saveAsTable('test')

In [ ]:
train = spark.table('train')
test = spark.table('test')

## Model Training
In MML Spark the LightGBM implementation for binary classification is invoked using the LightGBMClassifier class and specifying the objective as 'binary'. In this instance the occurrence of positive labels is quite low, so setting the isUnbalance flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key hyper-parameters \[5\] for training a LightGBM classifier on Spark
- numLeaves: the number of leaves in each tree
- numIterations: the number of iterations to apply boosting
- learningRate: the learning rate for training across trees
- featureFraction: the fraction of features used for training a tree
- earlyStoppingRound: round at which early stopping can be applied to avoid overfitting

In [14]:
NUM_LEAVES = 32
NUM_ITERATIONS = 10
LEARNING_RATE = 0.15
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 20

In [15]:
lgbm = LightGBMClassifier(
    labelCol='label',
    featuresCol='features',
    objective='binary',
    isUnbalance=True,
    boostingType='gbdt',
    boostFromAverage=True,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND,
)

### Model Training and Evaluation

In [16]:
model = lgbm.fit(train)
predictions = model.transform(test)
predictions.cache.count()

In [17]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("accuracy")
)

evaluator.transform(predictions).show()

+---------------+------------------+
|evaluation_type|          accuracy|
+---------------+------------------+
| Classification|0.7075339448071455|
+---------------+------------------+



### Model Tuning

MML Spark supports hyper-parameter tuning from a specified space of parameters which can be randomly sampled (or sampled from a grid of options) from continuous or discrete ranges of values. TuneHyperparameters can apply n-fold cross-validation with the given evaluation metric to more robustly identify the best set of parameters to use for the given model. 

In [ ]:
NUM_FOLDS = 3
NUM_RUNS = 10

In [ ]:
params = (
    HyperparamBuilder()
    .addHyperparam(lgbm, lgbm.learningRate, RangeHyperParam(0.001, 1.0))
    .addHyperparam(lgbm, lgbm.numIterations, RangeHyperParam(10, 100))
    .addHyperparam(lgbm, lgbm.numLeaves, DiscreteHyperParam([32, 64, 128]))
).build()
paramSpace = RandomSpace(params).space()

tuner = TuneHyperparameters(
    evaluationMetric="AUC", 
    models=[lgbm], 
    numFolds=NUM_FOLDS,
    numRuns=NUM_ROUNDS, 
    parallelism=1,
    paramSpace=paramSpace, 
    seed=42
)

best_model = tuner.fit(train)

In [ ]:
predictions = best_model.transform(test)
evaluator.transform(predictions).show()

## Model Saving and Loading
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [34]:
with TemporaryDirectory() as tmp:
    save_file = os.path.join(tmp, r'finished.model')
    pipeline = PipelineModel(stages=[feature_processor, model])
    pipeline.save(save_file)
    loaded_pipeline = PipelineModel.load(save_file)

In [36]:
# Test the loaded model on raw data
predictions = loaded_pipeline.transform(raw_data.limit(1000))
evaluator.transform(predictions).show()

+---------------+--------+
|evaluation_type|accuracy|
+---------------+--------+
| Classification|   0.739|
+---------------+--------+



## Model Deployment
MML Spark provides an easy way to quickly spin up a server to deploy trained models built on top of Spark Streaming DataFrames. In this example the server reads a request, parses it to the same input as the original raw data and applies feature processing then computes the probability of engagement given the user and item features provided. This probability is written back as a response to the original request.<br><br>
Content-based personalization can be accomplished by leveraging this engagement prediction service as the key machine-learning component inside a larger system. To personalize content for a user, a set of items is selected for evaluation and item-content features are extracted for each. These item features can be combined with the user features for each user-item combination and sent to the engagement prediction service which evaluates the probability that a user will engage with each item. The probability can be used to rank the items and select the top-k desired results.

In [59]:
# Define spark serving input
input_df = (
    spark.readStream.server()
    .address("localhost", 8089, "predict")
    .load()
    .parseRequest(raw_data.schema)
)

# Process features and reply with the probability of engagement
get_p_eng = udf(lambda p: float(p.toArray()[1]))
output_df = (
    loaded_pipeline.transform(input_df)
    .withColumn("p_eng", get_p_eng(col("probability")))
    .makeReply("p_eng")
)

# Define spark serving output and start server
checkpoint = TemporaryDirectory()
server = (
    output_df.writeStream.server()
    .replyTo("predict")
    .queryName("prediction")
    .option("checkpointLocation", "file://{}".format(checkpoint.name))
    .start()
)

In [60]:
server.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [61]:
query = raw_data.limit(1).collect()[0].asDict()
r = requests.post(json=query, url="http://localhost:8089/predict")
print("Response {}".format(r.text))

Response 0.7075531769538851


In [62]:
# Cleanup
server.stop()
checkpoint.cleanup()
server.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}